In [1]:
import numpy as np
import os
import glob
import math
import nibabel as nib
import cv2

from pycimg import CImg
from skimage import measure
from skimage import morphology
from scipy import ndimage
from scipy.spatial import ConvexHull


In [8]:
fname = './Images/IMG_0010.nii.gz'

foo = nib.load(fname).get_fdata()

print(foo.shape)
print(np.min(foo))
print(np.max(foo))
# CImg(foo).display();


(512, 512, 277)
-3024.0
3071.0


In [3]:
labels,_ = ndimage.label(foo)

if np.sum(labels) > 0:
    props = measure.regionprops(labels)
    ids,counts = np.unique(labels,return_counts=True)
    
    for i,c,regionProp in zip(ids[1:],counts[1:],props):   #bez tła

        voxels = np.where(labels==i)
        voxels = np.asarray(voxels)
        voxels = np.swapaxes(voxels,0,1)
        chull = ConvexHull(voxels,qhull_options = 'QJ')

        dum = np.zeros(labels.shape,dtype=np.float32)
        dum[labels==i] = 1
        verts, faces, _, _ = measure.marching_cubes(dum, 0.0)
        surf = measure.mesh_surface_area(verts, faces)

        print(fname,i, \
                surf, \
                math.pow(3*c/4/math.pi,1/3)/math.pow(surf/4/math.pi,1/2), \
                regionProp.area, \
                regionProp.inertia_tensor_eigvals[0], \
                regionProp.inertia_tensor_eigvals[1], \
                regionProp.inertia_tensor_eigvals[2], \
                regionProp.major_axis_length, regionProp.minor_axis_length, \
                regionProp.bbox_area, \
                regionProp.equivalent_diameter, \
                chull.volume,chull.area)


./Images/IMG_0001.nii.gz 1 92.9691390991211 74.48997803572934 34839793.0 43706.322276218474 23328.7380401739 23325.229234253642 661.1341095582554 171.68706993274324 34865152.0 405.2213457876902 34467973.68328954 792050.0172917995
